# 생활인구 데이터 홈페이지
https://data.seoul.go.kr/dataVisual/seoul/seoulLivingPopulation.do

In [26]:
from selenium import webdriver
import time
import folium
import os
import json
import pandas as pd
import numpy as np

In [8]:
date = '1123'

In [9]:
data_path = '/home/zimin/datasets/seoul/'
geo_path = data_path + 'skorea_submunicipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))
geo_str['features'] = [i for i in geo_str['features'] if i['properties']['code'][:2] == '11']

In [10]:
inner = pd.read_csv(os.path.join(data_path, 'pre_INNER_PEOPLE_2019' + date + '.csv'), index_col=0)

In [11]:
inner['행정동코드'] = inner['행정동코드'].apply(str)

In [12]:
inner

,기준일ID,시간대구분,행정동코드,거주지 자치구 코드,행정동이름,자치구이름,총생활인구수,110,115,120,...,235,240,245,250,255,260,265,270,275,300
0,20191123,0,1101072,11110,청운효자동,종로구,514.3462,*,21,74,...,23,19,21,26,8,17,7,17,11,28
1,20191123,0,1101072,11140,청운효자동,중구,22.1188,*,*,*,...,*,*,*,*,*,*,*,*,*,*
2,20191123,0,1101072,11170,청운효자동,용산구,49.5570,*,*,*,...,*,*,*,*,*,*,*,*,8,*
3,20191123,0,1101072,11200,청운효자동,성동구,33.9668,*,*,*,...,*,*,*,*,*,*,*,*,*,*
4,20191123,0,1101072,11215,청운효자동,광진구,14.9328,*,*,*,...,*,*,*,*,*,*,*,*,*,*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252565,20191123,23,1125071,11620,둔촌2동,관악구,28.4698,*,*,*,...,*,*,*,*,*,*,*,*,*,*
252566,20191123,23,1125071,11650,둔촌2동,서초구,17.1671,*,*,*,...,*,*,*,*,4,*,*,*,*,*
252567,20191123,23,1125071,11680,둔촌2동,강남구,77.8158,*,*,*,...,*,*,*,5,*,*,*,*,*,*
252568,20191123,23,1125071,11710,둔촌2동,송파구,364.7837,6,7,9,...,*,14,18,7,16,9,9,*,16,5


In [35]:
when = 20
big_where = '은평구'
who = '225'

In [36]:
check = inner[(inner['시간대구분'] == when) & (inner['자치구이름'] == big_where)][['행정동이름','행정동코드','자치구이름',who]]

In [37]:
check

,행정동이름,행정동코드,자치구이름,225
210396,청운효자동,1101072,은평구,23
210421,사직동,1101053,은평구,27
210446,삼청동,1101054,은평구,14
210471,부암동,1101055,은평구,9
210496,평창동,1101056,은평구,28
...,...,...,...,...
220846,성내2동,1125066,은평구,*
220871,성내3동,1125067,은평구,*
220896,길동,1125074,은평구,*
220921,둔촌1동,1125070,은평구,*


In [38]:
check[who] = check[who].replace('*',0)
check[[who]] = check[[who]].astype(int)

In [39]:
check['sum'] = check[[who]].sum(axis=1)

In [40]:
check['행정동코드'] = check['행정동코드'].apply(str)

In [45]:
seoul_map = folium.Map(location =[37.5502, 126.982], zoom_start=10.5, tiles = 'cartodbpositron')
seoul_map.choropleth(geo_data = geo_str,
                    data = check,
                    columns = ['행정동코드', 'sum'],
                    fill_color = 'PuRd',
                    key_on = 'properties.code',
                    highlight = True,
                    fill_opacity = 0.5, line_opacity = 1,
                    legend_name = 'good_boy')

title_html = '''
             <h3 align="center" style="font-size:20px"><b>{}</b></h3>
             '''.format(str(when)+ '시 ' + who + big_where)
seoul_map.get_root().html.add_child(folium.Element(title_html))

In [47]:
delay=1
fn=str(when)+ '시 ' + who + big_where +'.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd() + '/html_data',mapfile=fn)
seoul_map.save('./html_data/' + fn)
browser = webdriver.Chrome('/usr/bin/chromedriver')
browser.get(tmpurl)
#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('./screenshot/map.png')
browser.quit()